# Titanic Survival Prediction
**Author:** Kasim Morsel  
**Objective:** Predict Titanic passenger survival using machine learning (EDA → Feature Engineering → Model Comparison).  
**Tools:** Python, Pandas, NumPy, Matplotlib, Seaborn, Scikit-learn


In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Display settings
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')


In [4]:
# Load data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Preview data
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# Shape of datasets
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

# Info on columns & data types
train_df.info()

# Missing values (sorted highest → lowest)
train_df.isna().sum().sort_values(ascending=False)

# Target variable distribution
train_df['Survived'].value_counts(normalize=True)


In [ ]:
# Survival count
sns.countplot(data=train_df, x='Survived')
plt.title("Survival Count")
plt.show()

# Passenger class vs survival
sns.countplot(data=train_df, x='Pclass', hue='Survived')
plt.title("Survival by Passenger Class")
plt.show()

# Sex vs survival
sns.countplot(data=train_df, x='Sex', hue='Survived')
plt.title("Survival by Sex")
plt.show()

# Age distribution
sns.histplot(data=train_df, x='Age', bins=30, kde=True, hue='Survived', multiple='stack')
plt.title("Age Distribution by Survival")
plt.show()

# Fare distribution by survival
sns.boxplot(data=train_df, x='Survived', y='Fare')
plt.title("Fare vs Survival")
plt.show()

# Embarked port vs survival
sns.countplot(data=train_df, x='Embarked', hue='Survived')
plt.title("Survival by Embarkation Port")
plt.show()
